# Imports

In [12]:
import os
import shutil

from PAL import *
engine = Redshift_Connection('ssopariwalla')

In [6]:
schema = account_schema_mapping['MTS']

# Pull OI Alerts

In [39]:
query = f"""
SELECT DISTINCT
    event_table.event_id,
    vehicle_meta_data.vehicle_id,
    event_type,
    event_table.unique_id,
    event_table.pressure_date,
    CASE
        WHEN event_status.status = 'CLOSED' THEN event_status.ts_created
        ELSE NULL
    END as ts_closed
FROM 
    {schema}.event_table INNER JOIN {schema}.event_status
        ON event_table.event_id = event_status.event_id
    INNER JOIN {schema}.meta_data
        ON meta_data.unique_id = event_table.unique_id
    INNER JOIN {schema}.vehicle_meta_data
        ON vehicle_meta_data.vehicle_id = meta_data.vehicle_id
WHERE
    event_type = 'OI'
    AND event_table.pressure_date <= '2025-04-28 17:00'
    AND event_status.event_status_id IN (SELECT MAX(event_status_id) FROM {schema}.event_status GROUP BY event_id)
    AND meta_data.id IN (SELECT MAX(id) FROM {schema}.meta_data WHERE active = 1 GROUP BY unique_id)
ORDER BY
    event_table.event_id ASC
"""
OI_alerts = pd.read_sql(query,engine)

In [40]:
OI_alerts

,event_id,vehicle_id,event_type,unique_id,pressure_date,ts_closed
0,3259525,11510,OI,20971_819501EC,2025-04-24 12:28:45,NaT
1,3259765,4264,OI,4564_2D5755,2025-04-24 18:10:06,2025-04-24 19:01:28
2,3260012,4085,OI,4202_2D66FA,2025-04-25 11:33:58,2025-04-26 14:46:28
3,3260051,4264,OI,4564_2D5755,2025-04-25 12:26:12,2025-04-25 13:01:36
4,3260151,4212,OI,4486_2E09BE,2025-04-25 14:11:27,2025-04-25 15:01:26
5,3260155,4190,OI,3762_2A9655,2025-04-25 14:00:31,2025-04-25 16:19:19
6,3260280,4190,OI,3762_2A9655,2025-04-25 16:10:09,2025-04-25 19:16:28
7,3260301,4264,OI,4564_2D5755,2025-04-25 17:45:09,2025-04-25 18:16:25
8,3260302,4264,OI,4564_744860,2025-04-25 17:51:12,2025-04-25 18:16:25
9,3260307,4264,OI,4564_2D5755,2025-04-25 17:45:09,2025-04-25 18:31:36


# Plot Each OI Alert

In [10]:
os.makedirs('OI Alert Plots', exist_ok=True)


In [36]:
for index, row in OI_alerts.iterrows():
    query = f"""
    SELECT
        pressure_date,
        sensor_data.unique_id,
        set_point,
        pressure,
        speed
    FROM
        {schema}.sensor_data INNER JOIN {schema}.meta_data
            ON sensor_data.meta_data_id = meta_data.id
    WHERE
        sensor_data.unique_id = '{row['unique_id']}'
        AND pressure_date >= '{row['pressure_date'] - pd.Timedelta(hours=48)}'
        AND pressure_date <= '{row['pressure_date'] + pd.Timedelta(hours=48)}'
    ORDER BY
        pressure_date ASC
    """
    sensor_data = pd.read_sql(query, engine)

    # Create the plot
    fig = go.Figure()

    # Add pressure line (solid blue)
    fig.add_trace(go.Scatter(
        x=sensor_data['pressure_date'],
        y=sensor_data['pressure'],
        mode='lines',
        name='Pressure',
        line=dict(color='blue', width=2)
    ))

    # Add set_point line (dashed black)
    fig.add_trace(go.Scatter(
        x=sensor_data['pressure_date'],
        y=sensor_data['set_point'],
        mode='lines',
        name='Set Point',
        line=dict(color='black', width=2, dash='dash')
    ))

    # Add vertical red line at the alert's pressure_date
    fig.add_trace(go.Scatter(
        x=[row['pressure_date'], row['pressure_date']],
        y=[0, 145],
        mode='lines',
        name='Alert Time',
        line=dict(color='red', width=2, dash='dot')
    ))

    if not pd.isna(row['ts_closed']):
        # Add vertical red line at the alert's pressure_date
        fig.add_trace(go.Scatter(
            x=[row['ts_closed'], row['ts_closed']],
            y=[0, 145],
            mode='lines',
            name='Alert Time',
            line=dict(color='red', width=2)
        ))

    # Update layout
    fig.update_layout(
        title=f"Alert {row['event_id']}",
        xaxis_title="Pressure Date",
        yaxis_title="Pressure (PSI)",
        template="simple_white",
        yaxis=dict(range=[0, 140]),
        legend=dict(x=0, y=1)
    )

    fig.write_html(f"OI Alert Plots/Alert {row['event_id']}.html")


shutil.make_archive('OI_alert_plots', 'zip', 'OI Alert Plots')


'/home/sagemaker-user/SaroshSopariwalla/04_28_2025 Validating OI Alerts/OI_alert_plots.zip'